<a href="https://colab.research.google.com/github/nisha1365/Sanofi-POC/blob/main/HCP_v_s_Non_HCP_ROberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv("/content/my_data.csv")
df

,Unnamed: 0,tweet_id,external_twitter_id,text,author_id,created_at,lang,referenced_tweets_type,referenced_tweets_id,infos,association_tweet_id,association_scope_id
0,0,1030258,1.684840e+18,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,9.050890e+17,28-07-2023 08:18,en,0,0.000000e+00,{},1030258,10
1,1,1030186,1.684590e+18,"Roche Drops Hemophilia A Candidate, Prepares f...",7.638583e+07,27-07-2023 15:51,en,0,0.000000e+00,{},1030186,10
2,2,1030263,1.684810e+18,Roche moves one Spark hemophilia gene therapy ...,1.481280e+18,28-07-2023 06:34,en,0,0.000000e+00,{},1030263,10
3,3,1030274,1.684720e+18,On #WorldHepatitisDay we put middle finger up ...,1.145270e+18,28-07-2023 00:27,en,0,0.000000e+00,{},1030274,10
4,4,1030288,1.684660e+18,"One of our newest internship programs, launche...",2.112887e+08,27-07-2023 20:13,en,3,1.684630e+18,{},1030288,10
...,...,...,...,...,...,...,...,...,...,...,...,...
29798,29798,1683839551645814784,NaN,"Hear more from Mosi Williams, Pys.D, on the la...",1.479181e+18,2023-07-25 14:00:07,en,0,0.000000e+00,{},1683839551645814784,10
29799,29799,1683938283003183104,NaN,"Was discussing hemophilia with an intelligent,...",2.760856e+08,2023-07-25 20:32:27,en,0,0.000000e+00,{},1683938283003183104,10
29800,29800,1683861079674195968,NaN,The affected &amp; infected have waited decade...,1.104181e+08,2023-07-25 15:25:40,en,3,1.683849e+18,{},1683861079674195968,10
29801,29801,1683918306082955264,NaN,"@MohmdBio Sickle cell anemia, Thalassemia, Hem...",1.179873e+18,2023-07-25 19:13:04,en,2,1.683918e+18,{},1683918306082955264,10


In [2]:
columns_to_keep = ['author_id','tweet_id','text']

new_df = df[columns_to_keep]
new_df.head()

,author_id,tweet_id,text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...
4,2.112887e+08,1030288,"One of our newest internship programs, launche..."


In [3]:
!pip install tweet-preprocessor

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# import tweet preprocessor
import preprocessor as p

import nltk
import gensim
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords


In [5]:
# Customizing the preprocessor to exclude removing hashtags since they are valuable as a rich information
p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION,p.OPT.SMILEY,p.OPT.NUMBER )

#forming a separate feature for cleaned tweets
for i,v in enumerate(new_df['text']):
    new_df.loc[i,'new_text'] = p.clean(v)

new_df.head()

<ipython-input-5-97c3e15aa739>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[i,'new_text'] = p.clean(v)


,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks Challenge #AxioActive On the th of Augus...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...","Roche Drops Hemophilia A Candidate, Prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,Roche moves one Spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,On #WorldHepatitisDay we put middle finger up ...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...","One of our newest internship programs, launche..."


In [6]:

#REMOVE ONLY THE '#'NOT THE WORD AFTER
def remove_hashtag_sign(text):
    text = re.sub(r'#', '', text)
    return text

new_df['new_text'] = new_df['new_text'].apply(lambda x:remove_hashtag_sign(x))

<ipython-input-6-95cc9cf31c2a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_text'] = new_df['new_text'].apply(lambda x:remove_hashtag_sign(x))


In [7]:
#Remove extra white spaces, punctuation and apply lower casing
new_df['new_text'] = new_df['new_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
new_df.head()


<ipython-input-7-9b51ea083bf1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['new_text'] = new_df['new_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
<ipython-input-7-9b51ea083bf1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_text'] = new_df['new_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks challenge axioactive on the th of august...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...",roche drops hemophilia a candidate prepares fo...
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,roche moves one spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,on worldhepatitisday we put middle finger up t...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...",one of our newest internship programs launched...


In [8]:
new_df.shape

(29803, 4)

In [9]:

new_df_subset1 = new_df.head(1000)

In [10]:
new_df_subset1


,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks challenge axioactive on the th of august...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...",roche drops hemophilia a candidate prepares fo...
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,roche moves one spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,on worldhepatitisday we put middle finger up t...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...",one of our newest internship programs launched...
...,...,...,...,...
995,1.935531e+08,1033001,IU researchers uncover mysteries behind immune...,iu researchers uncover mysteries behind immune...
996,1.409967e+08,1033007,@HemoIX @JonRob08 @ProfMakris @NHF_Hemophilia ...,im a patient
997,1.498170e+18,1032973,IU researchers uncover mysteries behind immune...,iu researchers uncover mysteries behind immune...
998,2.280750e+09,1032974,"@MrDelicious13 @caseyboguslaw Jesus, we’ve got...",jesus weve got that back woods swamp dna then ...


In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="openai-community/roberta-base-openai-detector")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("openai-community/roberta-base-openai-detector")
model = AutoModelForSequenceClassification.from_pretrained("openai-community/roberta-base-openai-detector")

Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
import torch

In [14]:
# Provide information about Health Care Providers
hcp_info = "Health care providers are professionals such as doctors, nurses, and other medical personnel, They play a crucial role in diagnosing, treating and supporting individual's health needs within medical settings, classify the tweets as HCP(1) or Non-HCP(0) by understanding the context."

# Create lists to store model predictions and probabilities
model_predictions = []
model_probabilities = []

# Iterate through each tweet
for i, text in enumerate(new_df_subset1['new_text']):
    # Display the tweet
    print(f"\nTweet {i + 1}:\n{text}")

    # Tokenize the tweet with information about HCP
    tokenized_text = tokenizer.encode(hcp_info + ' ' + text, return_tensors="pt", max_length=512, truncation=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(tokenized_text)  # No need to add an extra dimension for batch size

    # Assuming binary classification
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()

    # Append model's predictions and probabilities to the lists
    model_predictions.append(predicted_label)
    model_probabilities.append(probabilities[0, 1].item())

# Add model's predictions and probabilities to the DataFrame
new_df_subset1['model_prediction'] = model_predictions
new_df_subset1['model_probability_hcp'] = model_probabilities

# Display or save the DataFrame with model predictions and probabilities
print(new_df_subset1)


Tweet 1:
peaks challenge axioactive on the th of august some of the axiologik teammates will be taking on the yorkshire peaks within hours to raise significant funds for the haemophilia society all donations are truly appreciated 

Tweet 2:
roche drops hemophilia a candidate prepares for the impacts of ira

Tweet 3:
roche moves one spark hemophilia gene therapy into phiii cuts another

Tweet 4:
on worldhepatitisday we put middle finger up to australias corrupt health officials whove let thousands die due to their coverup of the infectedbloodscandal which has only been possible through complicit journalists who support genocide haemophilia contaminatedblood

Tweet 5:
one of our newest internship programs launched in raquel is researching how high cost hemophilia therapies can be optimally managed through specialtypharmacy nationalinternday amcpfdn

Tweet 6:
delighted and honored to see our paper published with a commentary by dr kate pratt and featured in the latest july issue of hemop

<ipython-input-14-851ba9676a81>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_subset1['model_prediction'] = model_predictions
<ipython-input-14-851ba9676a81>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_subset1['model_probability_hcp'] = model_probabilities


In [15]:
new_df_subset1['model_prediction'].value_counts()

1    866
0    134
Name: model_prediction, dtype: int64